This notebook joins 311 social distancing complaints to Parks Properties using both spatial joins as well as fuzzy text matches using the 'description' field. Output is written to Google Sheets as number of complaints per park. 

Created by Julie Tsitron (julie.tsitron@parks.nyc.gov) in April 2020. Updated on 5/14/2021 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1">imports</a></span></li><li><span><a href="#authentication" data-toc-modified-id="authentication-2">authentication</a></span></li><li><span><a href="#worksheets-created-for-output" data-toc-modified-id="worksheets-created-for-output-3">worksheets created for output</a></span></li><li><span><a href="#parks-data-(can-also-pull-this-from-Open-Data-)" data-toc-modified-id="parks-data-(can-also-pull-this-from-Open-Data-)-4">parks data (can also pull this from <strong><a href="https://data.cityofnewyork.us/Recreation/Parks-Properties/enfh-gkve" target="_blank">Open Data</a></strong> )</a></span></li><li><span><a href="#sample-311-data-(looking-at-social-distancing-complaints-in-this-example)-This-is-also-available-on--Open-Data" data-toc-modified-id="sample-311-data-(looking-at-social-distancing-complaints-in-this-example)-This-is-also-available-on--Open-Data-5">sample 311 data (looking at social distancing complaints in this example) This is also available on  <strong><a href="https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9" target="_blank">Open Data</a></strong></a></span><ul class="toc-item"><li><span><a href="#location-field" data-toc-modified-id="location-field-5.1">location field</a></span></li><li><span><a href="#look-at-previous-day's-complaints" data-toc-modified-id="look-at-previous-day's-complaints-5.2">look at previous day's complaints</a></span></li></ul></li><li><span><a href="#helper-functions-for-joining-311-complaints-to-parks" data-toc-modified-id="helper-functions-for-joining-311-complaints-to-parks-6">helper functions for joining 311 complaints to parks</a></span><ul class="toc-item"><li><span><a href="#spatial-joins" data-toc-modified-id="spatial-joins-6.1">spatial joins</a></span></li><li><span><a href="#'fuzzy'-text-match-joins" data-toc-modified-id="'fuzzy'-text-match-joins-6.2">'fuzzy' text match joins</a></span></li><li><span><a href="#concatenate-all-matches" data-toc-modified-id="concatenate-all-matches-6.3">concatenate all matches</a></span></li></ul></li><li><span><a href="#join-to-parks" data-toc-modified-id="join-to-parks-7">join to parks</a></span></li><li><span><a href="#cells-below-are-for-writing-output-to-google-sheet-(disregard)" data-toc-modified-id="cells-below-are-for-writing-output-to-google-sheet-(disregard)-8">cells below are for writing output to google sheet (disregard)</a></span></li></ul></div>

# imports

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import gc
import datetime
import sys
import re
import geopandas as gpd
from geopandas import GeoDataFrame, GeoSeries
# import google.auth
from gspread import Client
from gspread_dataframe import set_with_dataframe

sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions_old import create_assertion_session
from IPM_Shared_Code_public.Python.geo_functions import read_geosql  ## this function allows us to read in spatial data directly from SQL Server into a GeoDataFrame
from IPM_Shared_Code_public.Python.utils import get_config

# authentication

In [2]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
data_parks = config['srv']['data_parks']
dwh = config['db']['dwh']
cred_file = config['google']['path_to_file']

In [3]:
con = pyodbc.connect('Driver={' + driver + '};Server=' + server +
                     ';Database=' + dwh + ';Trusted_Connection=Yes;')
con_data_parks = pyodbc.connect('Driver={' + driver + '};Server=' +
                                data_parks + ';Database=' + dwh +
                                ';Trusted_Connection=Yes;')

In [4]:
scope = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive'
]

In [5]:
session = create_assertion_session(cred_file, scope)

In [6]:
gc = Client(None, session)

# worksheets created for output

In [7]:
wks = gc.open("Daily_311_SRs_at_Parks")
wks2 = gc.open("Daily 311 SR Hotspots at Parks")

# parks data (can also pull this from __[Open Data](https://data.cityofnewyork.us/Recreation/Parks-Properties/enfh-gkve)__ )

In [9]:
sql = 'select * from [DWH].[dbo].[TBL_PARKSGIS_PROPERTY]'
parks = read_geosql(sql, con_data_parks, geom_raw='Shape', geom_col='geometry')

# sample 311 data (looking at social distancing complaints in this example) This is also available on  __[Open Data](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)__ 

In [10]:
sql_311='''
SELECT * FROM [DWH].[dbo].[tbl_311_social_distancing]
'''

In [11]:
SD_311 = pd.read_sql(sql_311, con)

In [12]:
SD_311.sort_values(by='open_datetime').tail()

,sr_num,location_type,description_text,descriptor_1,agency,open_datetime,closed_datetime,resolution,resolution_action_updated_dt,status,...,street,zip,borough,precinct,patrolboroughcommand,park_facility_name,park_borough,lat,lon,shape
63941,311-06208474,Park,Men’s from the men shelter in the park play ar...,Social Distancing,DPR,2021-05-09 08:34:07,2021-05-09 09:15:36,Upon inspection the reported condition was not...,2021-05-09 13:15:39,Closed,...,BAISLEY POND PARK,11434,QUEENS,Precinct 113,Queens South,Baisley Pond Park,QUEENS,40.676156,-73.785453,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00b\xd...
63942,311-06230854,Park,THERE ARE MULTIPLE PEOPLE RUNNING ON THE JOGGI...,Social Distancing,DPR,2021-05-11 19:10:41,2021-05-11 19:21:07,No action was taken because the agency determi...,2021-05-11 23:21:11,Closed,...,None,None,BROOKLYN,Unspecified,Unspecified,Prospect Park,BROOKLYN,NaN,NaN,None
63943,311-06232321,Park,Resident rpts basketball players at location a...,Social Distancing,DPR,2021-05-11 22:06:03,2021-05-12 00:26:16,NYC Parks has reviewed the reported condition ...,2021-05-12 04:26:19,Closed,...,32 AVENUE,11354,QUEENS,Precinct 109,Queens North,Bowne Park,QUEENS,40.769851,-73.809152,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00r\xa...
63945,311-06246159,Park,Hi. Im not sure where to file this but the bil...,Social Distancing,DPR,2021-05-13 11:18:14,2021-05-13 14:03:43,Upon inspection the reported condition was not...,2021-05-13 18:03:47,Closed,...,WEST 150 STREET,10039,MANHATTAN,Precinct 32,Manhattan North,Unspecified,MANHATTAN,40.825535,-73.938470,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00~\x8...
63944,311-06249675,Park,These people are back at it. I just saw them b...,Social Distancing,DPR,2021-05-13 17:09:40,2021-05-13 20:05:42,NYC Parks couldn’t respond to your complaint b...,2021-05-14 01:48:21,Closed,...,EAST 137 STREET,10454,BRONX,Precinct 40,Bronx,United We Stand Garden,BRONX,40.804652,-73.914497,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00b\xc...


## location field

In [13]:
list(SD_311.sort_values(by='open_datetime').location_type.unique())

['Park/Playground',
 'Street/Sidewalk',
 'Store/Commercial',
 'Residential Building/House',
 None,
 'Park',
 'Street/Curbside']

## look at previous day's complaints

In [14]:
# Look at data from 2020 (more interesting):
today = datetime.datetime.today() - datetime.timedelta(days=365)

In [15]:
Year = today.year
Month = today.month
Day = today.day
Hour = 2
Min = 0
Sec = 0

In [16]:
dayOfInterest_st = pd.Timestamp(
    year=Year, month=Month, day=Day, hour=Hour, minute=Min,
    second=Sec) - datetime.timedelta(days=1)
dayOfInterest_end = pd.Timestamp(
    year=Year, month=Month, day=Day, hour=Hour, minute=Min,
    second=Sec) - datetime.timedelta(days=0)

In [17]:
SD_311_day = SD_311[(SD_311['open_datetime'] > dayOfInterest_st) & (
    SD_311['open_datetime'] <= dayOfInterest_end)].sort_values(
        by='open_datetime')

In [18]:
SD_311_day.head()

,sr_num,location_type,description_text,descriptor_1,agency,open_datetime,closed_datetime,resolution,resolution_action_updated_dt,status,...,street,zip,borough,precinct,patrolboroughcommand,park_facility_name,park_borough,lat,lon,shape
31939,311-02452843,Store/Commercial,K c & k beauty supply \n234-01 Merrick Blvd \n...,Social Distancing,NYPD,2020-05-13 02:13:28,2020-05-13 02:21:34,The Police Department responded to the complai...,2020-05-13 02:19:00,Closed,...,MERRICK BOULEVARD,11422,QUEENS,Precinct 105,Queens South,Unspecified,QUEENS,40.674450,-73.735917,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xdf...
32489,311-02452868,Street/Sidewalk,SOCIAL DISTANCING VIOLATION.,Social Distancing,NYPD,2020-05-13 02:24:03,2020-05-13 02:32:09,The Police Department responded to the complai...,2020-05-13 02:32:00,Closed,...,10 AVENUE,10019,MANHATTAN,Precinct 18,Manhattan South,Unspecified,MANHATTAN,40.765933,-73.990877,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x002\x1...
32487,311-02452915,Park/Playground,A pack of rare mix breed pit bulls are running...,Social Distancing,NYPD,2020-05-13 02:26:46,2020-05-13 02:34:53,The Police Department responded to the complai...,2020-05-13 02:34:00,Closed,...,BAISLEY BOULEVARD,11434,QUEENS,Precinct 113,Queens South,Unspecified,QUEENS,40.682430,-73.771061,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\x86...
32303,311-02452954,Street/Sidewalk,Sidewalk fruit vendor not following social dis...,Social Distancing,NYPD,2020-05-13 02:48:51,2020-05-13 03:07:22,The Police Department responded to the complai...,2020-05-13 03:07:00,Closed,...,LENOX AVENUE,10037,MANHATTAN,Precinct 32,Manhattan North,Unspecified,MANHATTAN,40.813106,-73.941577,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xcc...
31854,311-02452984,Street/Sidewalk,I was walking on Sterling place and Nostrand A...,Social Distancing,NYPD,2020-05-13 03:12:05,2020-05-13 03:19:58,The Police Department responded to the complai...,2020-05-13 03:19:00,Closed,...,NOSTRAND AVENUE,11216,BROOKLYN,Precinct 77,Brooklyn North,Unspecified,BROOKLYN,40.672530,-73.950251,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00Du.A...


# helper functions for joining 311 complaints to parks

## spatial joins

In [19]:
def get_gdf_from_df(df, lat_col, lon_col):
    return gpd.GeoDataFrame(df[~pd.isnull(df[lat_col])],
                            geometry=gpd.points_from_xy(
                                df[~pd.isnull(df[lat_col])][lon_col],
                                df[~pd.isnull(df[lat_col])][lat_col]),
                            crs=('epsg:4326'))

In [20]:
def buffer(gdf, buffer_radius, lat_col):
    geom_buffered = []
    for i in gdf.geometry:
        geom_buffered.append(i.buffer(buffer_radius))
    crs = 'epsg:2236'
    buffered_gdf = gpd.GeoDataFrame(gdf[~pd.isnull(gdf[lat_col])],
                                    crs=crs,
                                    geometry=geom_buffered)
    return buffered_gdf

In [21]:
def get_spatial_match(buffered_df, parks_df):
    return gpd.sjoin(buffered_df, parks_df, op='intersects')

In [22]:
def filter_spatial_match(spatial_match_df, loc_col, non_park_locs_list,
                         cols_to_keep_list):
    return spatial_match_df[~spatial_match_df[loc_col].
                            isin(non_park_locs_list)][cols_to_keep_list]

In [23]:
def get_missed_desc(df, spatial_match_filtered_df):
    missing_descriptions_list = []
    groups = df.groupby(loc_col).groups.keys()
    if 'Park/Playground' in groups:
        for i in df.groupby(loc_col).get_group(
                'Park/Playground')['description_text'].values:
            if i not in spatial_match_filtered_df.description_text.values:
                print(i)
                missing_descriptions_list.append(i)
    return missing_descriptions_list

## 'fuzzy' text match joins

In [24]:
def match_park_text(park_df, parkname_col, pat1, pat2):
    return re.compile('({})'.format("|".join(
        list(park_df[(park_df[parkname_col] != pat1)
                     & (park_df[parkname_col] != pat2)]
             [parkname_col].str.lower().values))))

In [25]:
def match_any_text(pat):
    return re.compile(pat,re.UNICODE)

In [26]:
def find_parkname(row, descr_col, pat):
    if (re.search(pat, row[descr_col].lower())) != None:
        return re.search(pat, row[descr_col].lower()).groups()[0]

In [27]:
def get_missed_park_rows(gdf, desc_col, missed_desc_list, lat_col, loc_col):
    return gdf[
        (gdf[desc_col].isin(missed_desc_list))
        |
        ((pd.isnull(gdf[lat_col])) &
         (gdf[loc_col] == 'Park/Playground'))].copy()  

## concatenate all matches

In [28]:
def concat_spatial_and_text_matches(spatial_match_filtered_df,
                                    missed_park_rows_df):
    return pd.concat([
        spatial_match_filtered_df,
        missed_park_rows_df[~pd.isnull(missed_park_rows_df[parkname_col])]
    ],
                     sort=False)

# join to parks

In [30]:
sr_num_col = 'sr_num'
parkname_col = 'PropertyName'
desc_col = 'description_text'
lat_col = 'lat'
lon_col = 'lon'
loc_col = 'location_type'

cols_to_keep = [
    'sr_num', 'location_type', 'description_text', 'descriptor_1', 'agency',
    'open_datetime', 'closed_datetime', 'resolution',
    'resolution_action_updated_dt', 'status', 'bin', 'bbl', 'house_number',
    'street', 'zip', 'borough', 'precinct', 'patrolboroughcommand',
    'park_facility_name', 'park_borough', 'lat', 'lon', 'geometry',
    'index_right', 'PropertyName'
]

non_park_locs = [
    'Residential Building/House', 'Street/Sidewalk', 'Store/Commercial', ''
]

pat_park = match_park_text(parks, 'PropertyName', 'Park', 'Lot')

pat_mask = match_any_text(r'(mask|face covering)')

In [31]:
complaints_day_gdf = get_gdf_from_df(SD_311_day, lat_col, lon_col)

complaints_day_gdf= complaints_day_gdf.to_crs('epsg:2236')

In [32]:
complaints_day_gdf_buffered = buffer(complaints_day_gdf,100,'lat')

In [33]:
parks = parks.to_crs('epsg:2236')

In [34]:
complaints_day_gdf_buffered.head()

,sr_num,location_type,description_text,descriptor_1,agency,open_datetime,closed_datetime,resolution,resolution_action_updated_dt,status,...,zip,borough,precinct,patrolboroughcommand,park_facility_name,park_borough,lat,lon,shape,geometry
31939,311-02452843,Store/Commercial,K c & k beauty supply \n234-01 Merrick Blvd \n...,Social Distancing,NYPD,2020-05-13 02:13:28,2020-05-13 02:21:34,The Police Department responded to the complai...,2020-05-13 02:19:00,Closed,...,11422,QUEENS,Precinct 105,Queens South,Unspecified,QUEENS,40.674450,-73.735917,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xdf...,"POLYGON ((2671924.362 6028736.009, 2671923.881..."
32489,311-02452868,Street/Sidewalk,SOCIAL DISTANCING VIOLATION.,Social Distancing,NYPD,2020-05-13 02:24:03,2020-05-13 02:32:09,The Police Department responded to the complai...,2020-05-13 02:32:00,Closed,...,10019,MANHATTAN,Precinct 18,Manhattan South,Unspecified,MANHATTAN,40.765933,-73.990877,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x002\x1...,"POLYGON ((2598447.784 6056323.473, 2598447.302..."
32487,311-02452915,Park/Playground,A pack of rare mix breed pit bulls are running...,Social Distancing,NYPD,2020-05-13 02:26:46,2020-05-13 02:34:53,The Police Department responded to the complai...,2020-05-13 02:34:00,Closed,...,11434,QUEENS,Precinct 113,Queens South,Unspecified,QUEENS,40.682430,-73.771061,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\x86...,"POLYGON ((2661924.206 6030837.903, 2661923.724..."
32303,311-02452954,Street/Sidewalk,Sidewalk fruit vendor not following social dis...,Social Distancing,NYPD,2020-05-13 02:48:51,2020-05-13 03:07:22,The Police Department responded to the complai...,2020-05-13 03:07:00,Closed,...,10037,MANHATTAN,Precinct 32,Manhattan North,Unspecified,MANHATTAN,40.813106,-73.941577,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xcc...,"POLYGON ((2610726.830 6074629.665, 2610726.349..."
31854,311-02452984,Street/Sidewalk,I was walking on Sterling place and Nostrand A...,Social Distancing,NYPD,2020-05-13 03:12:05,2020-05-13 03:19:58,The Police Department responded to the complai...,2020-05-13 03:19:00,Closed,...,11216,BROOKLYN,Precinct 77,Brooklyn North,Unspecified,BROOKLYN,40.672530,-73.950251,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00Du.A...,"POLYGON ((2612461.542 6023164.598, 2612461.060..."


In [35]:
spatial_match = get_spatial_match(complaints_day_gdf_buffered,parks)

In [36]:
spatial_match.head()

,sr_num,location_type,description_text,descriptor_1,agency,open_datetime,closed_datetime,resolution,resolution_action_updated_dt,status,...,Waterfront,CreatedUser,CreatedDate,LastEditedUser,LastEditedDate,NYSAssembly,NYSSenate,USCongress,GlobalID,GDB_GEOMATTR_DATA
31882,311-02453438,Residential Building/House,THIS SHELTER IS NOT PRACTICING SOCIAL DISTANCI...,Social Distancing,NYPD,2020-05-13 07:35:58,2020-05-13 08:05:02,The Police Department responded to the complai...,2020-05-13 08:04:00,Closed,...,NO,None,None,NYCDPR\Alex.Butler,2021-01-29 18:02:05.0000000,86,33,15,BE14A4EC-F5BA-4711-8A55-A3F87D671B17,None
31947,311-02453533,None,The manager in the store and the employees hav...,Social Distancing,NYPD,2020-05-13 07:59:43,2020-05-13 08:07:47,The Police Department responded to the complai...,2020-05-13 08:07:00,Closed,...,NO,None,None,BEN.BLACKSHEAR,2020-07-21 16:53:04.0000000,"70, 71",30,13,2468B9EF-4D12-4E2C-9D62-31AAAECF8B58,None
32494,311-02457190,None,THEY ARE NOT SOCIAL DISTANCING AT LOCATION IN ...,Social Distancing,NYPD,2020-05-13 15:50:09,2020-05-13 16:31:26,The Police Department responded to the complai...,2020-05-13 16:31:00,Closed,...,NO,None,None,BEN.BLACKSHEAR,2020-07-21 16:53:04.0000000,"70, 71",30,13,2468B9EF-4D12-4E2C-9D62-31AAAECF8B58,None
31875,311-02459398,Store/Commercial,"Young adults hanging out drinking, smoking, li...",Social Distancing,NYPD,2020-05-13 20:11:32,2020-05-13 20:34:48,The Police Department responded to the complai...,2020-05-13 20:34:00,Closed,...,NO,None,None,BEN.BLACKSHEAR,2020-07-21 16:53:04.0000000,"70, 71",30,13,2468B9EF-4D12-4E2C-9D62-31AAAECF8B58,None
31984,311-02453620,Street/Sidewalk,14 unmasked crowd on park benches on Central P...,Social Distancing,NYPD,2020-05-13 08:19:55,2020-05-13 08:38:33,The Police Department responded to the complai...,2020-05-13 08:38:00,Closed,...,NO,None,None,ANTONIOS.MICHELAKIS,2020-12-14 21:02:04.0000000,"69, 68, 67, 75",29,"10, 13",9B51C8B1-5896-4A38-BAC1-FD5E6DFD83D6,None


In [37]:
len(spatial_match)

132

In [38]:
spatial_match_filtered = filter_spatial_match(spatial_match, loc_col,
                                              non_park_locs, cols_to_keep)

In [39]:
len(spatial_match_filtered)

73

In [40]:
missing_descriptions = get_missed_desc(SD_311_day, spatial_match_filtered)

A pack of rare mix breed pit bulls are running around the park ignoring social distancing guidelines. Leader of pack will respond to “Champ”.
Guys from the block, bbq and hanging out at the park across the street from 1149 Morris ave.
SOCCER TEAM  PLAYING IN THE PARK WITH 10 PEOPLE AND 2 ADULTS IN BIG BUSH PARK.
overcrowding in park area, no social distancing
WILLIAM F PASSANNANTA BALL FIELD
EVERY DAY THERE IS A CROWD IN THE SCHOOL YARD WHICH HAS A LOCK ON IT AROUND 3:30PM. THEY ARE PLAYING AND NOT PRACTICING SOCIAL DISTANCING. THE FENCE IS LOCKED SO THEY ARE GETTING IN ANOTHER WAY
A youth soccer team practices every day on the East River track (FDR/north of Houston). They take up a lot of space in the middle of the field and don't adhere to social distancing guidelines and endanger the people around them who are attempting to solo exercise. It's a large group of players, coaches, siblings, parents who get on the middle of the field and slowly take up more and more space.
Charles Park 

In [41]:
missed_park_rows = get_missed_park_rows(complaints_day_gdf, desc_col,
                                        missing_descriptions, lat_col, loc_col)

In [42]:
missed_park_rows.head()

,sr_num,location_type,description_text,descriptor_1,agency,open_datetime,closed_datetime,resolution,resolution_action_updated_dt,status,...,zip,borough,precinct,patrolboroughcommand,park_facility_name,park_borough,lat,lon,shape,geometry
32487,311-02452915,Park/Playground,A pack of rare mix breed pit bulls are running...,Social Distancing,NYPD,2020-05-13 02:26:46,2020-05-13 02:34:53,The Police Department responded to the complai...,2020-05-13 02:34:00,Closed,...,11434,QUEENS,Precinct 113,Queens South,Unspecified,QUEENS,40.682430,-73.771061,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\x86...,POINT (2661824.206 6030837.903)
32133,311-02453700,Park/Playground,"Guys from the block, bbq and hanging out at th...",Social Distancing,NYPD,2020-05-13 08:40:36,2020-05-13 09:18:08,The Police Department responded to the complai...,2020-05-13 09:18:00,Closed,...,10456,BRONX,Precinct 44,Bronx,Unspecified,BRONX,40.832143,-73.914748,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xc4...,POINT (2617497.162 6082175.677)
31976,311-02454070,Park/Playground,SOCCER TEAM PLAYING IN THE PARK WITH 10 PEOPL...,Social Distancing,NYPD,2020-05-13 09:39:57,2020-05-13 10:11:04,The Police Department responded to the complai...,2020-05-13 10:11:00,Closed,...,11377,QUEENS,Precinct 108,Queens North,Unspecified,QUEENS,40.740623,-73.897752,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\xd0...,POINT (2624922.064 6049179.293)
31975,311-02454278,Park/Playground,"overcrowding in park area, no social distancing",Social Distancing,NYPD,2020-05-13 10:15:16,2020-05-13 10:17:18,The Police Department responded to the complai...,2020-05-13 10:17:00,Closed,...,10014,MANHATTAN,Unspecified,Unspecified,Unspecified,MANHATTAN,40.729170,-74.010676,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00V\xf...,POINT (2593930.689 6042475.401)
32279,311-02454483,Park/Playground,WILLIAM F PASSANNANTA BALL FIELD\nEVERY DAY TH...,Social Distancing,NYPD,2020-05-13 10:36:40,2020-05-13 10:42:35,The Police Department responded to the complai...,2020-05-13 10:41:00,Closed,...,10012,MANHATTAN,Precinct 6,Manhattan South,Unspecified,MANHATTAN,40.727749,-74.001551,b'\xd7\x08\x00\x00\x01\x0c\x00\x00\x00\x00\x18...,POINT (2596503.872 6042159.688)


In [43]:
len(missed_park_rows)

19

In [44]:
if not missed_park_rows.empty:
    missed_park_rows[parkname_col] = missed_park_rows.apply(
        lambda row: find_parkname(row, desc_col, pat_park), axis=1)

In [45]:
if not missed_park_rows.empty:
    all_matches = concat_spatial_and_text_matches(
        spatial_match_filtered, missed_park_rows)
else:
    all_matches = spatial_match_filtered.copy()## This is the full list

In [46]:
counts = all_matches.groupby(parkname_col).count()[[sr_num_col]].sort_values(
    by=sr_num_col, ascending=False)

hotspots = counts[counts[sr_num_col] > 1].copy()

hotspots.rename(columns={sr_num_col: 'Social Distancing 311 SRs'},
                inplace=True)

In [47]:
counts

,sr_num
PropertyName,
Central Park,4
Van Cortlandt Park,3
Broadway Malls,3
Adam Clayton Powell Jr. Malls,2
Maria Hernandez Park,2
McKinley Park,2
Isham Park,2
Inwood Hill Park,2
Riverside Park,2


In [48]:
hotspots

,Social Distancing 311 SRs
PropertyName,
Central Park,4
Van Cortlandt Park,3
Broadway Malls,3
Adam Clayton Powell Jr. Malls,2
Maria Hernandez Park,2
McKinley Park,2
Isham Park,2
Inwood Hill Park,2
Riverside Park,2


In [49]:
hotspots = pd.merge(
    hotspots.reset_index(),
    all_matches[all_matches[parkname_col].isin(hotspots.index.values)][[
        'PropertyName',
        'precinct',
        'patrolboroughcommand',
        'sr_num',
        'open_datetime',
    ]],
    left_on='PropertyName',
    right_on='PropertyName').sort_values(
        by=['Social Distancing 311 SRs', 'PropertyName', 'open_datetime'],
        ascending=False).set_index('Social Distancing 311 SRs')

In [50]:
hotspots

,PropertyName,precinct,patrolboroughcommand,sr_num,open_datetime
Social Distancing 311 SRs,,,,,
4,Central Park,Precinct 24,Manhattan North,311-02460672,2020-05-13 22:54:12
4,Central Park,Precinct 22,Manhattan North,311-02460301,2020-05-13 22:09:18
4,Central Park,Precinct 22,Manhattan North,311-02458073,2020-05-13 17:34:20
4,Central Park,Precinct 22,Manhattan North,311-02457177,2020-05-13 15:47:39
3,Van Cortlandt Park,Precinct 50,Bronx,311-02458779,2020-05-13 18:56:13
3,Van Cortlandt Park,Precinct 50,Bronx,311-02458429,2020-05-13 18:19:44
3,Van Cortlandt Park,Precinct 50,Bronx,311-02456109,2020-05-13 13:49:52
3,Broadway Malls,Precinct 30,Manhattan North,311-02457841,2020-05-13 17:01:20
3,Broadway Malls,Precinct 24,Manhattan North,311-02454456,2020-05-13 10:29:40


# cells below are for writing output to google sheet (disregard)

In [91]:
todays_date = today - datetime.timedelta(days=1)

In [ ]:
wks.add_worksheet(
    'all matches ' + str(todays_date.date()),
    len(
        all_matches.sort_values(by='open_datetime').drop(
            ['index_right', 'shape'], axis=1, errors='ignore')) + 1,
    len(
        all_matches.sort_values(by='open_datetime').drop(
            ['index_right', 'shape'], axis=1, errors='ignore').columns.values))

In [ ]:
wks2.add_worksheet(
    'hotspots '+str(todays_date.date()),
    len(hotspots)+1,
    len(hotspots.reset_index().columns.values));

In [ ]:
Sheet1 = wks.sheet1

In [ ]:
hotspots_sheet = wks2.sheet1

In [ ]:
wks.del_worksheet(Sheet1);

In [ ]:
wks2.del_worksheet(hotspots_sheet);

In [ ]:
# set_with_dataframe(
#     wks.sheet1,
#     all_matches.sort_values(by='open_datetime').drop(['index_right'],
#                                                      axis=1))

In [ ]:
set_with_dataframe(
    wks.sheet1,
    all_matches.sort_values(by='open_datetime').drop(['index_right', 'shape'],
                                                     axis=1,errors='ignore'))

In [ ]:
all_matches

In [ ]:
set_with_dataframe(wks2.sheet1, hotspots.reset_index())

In [ ]:
all_matches.sort_values(by='open_datetime')